# Wide Resnetをためす

In [6]:
import shutil
import wide_resnet as wrn
from preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import pathlib
import os
import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score
import keras.callbacks as callbacks
from keras.utils import plot_model
from keras import backend as K
from keras import optimizers
import tensorflow as tf

## 画像をディレクトリに分割する

In [3]:
DATA_DIR = pathlib.Path('../data/')
LOG_DIR = pathlib.Path('../log/')

In [18]:
df = pd.read_csv(DATA_DIR.joinpath('train_master.tsv'), delimiter='\t')

In [19]:
df

,file_name,flag
0,train_0.tif,0
1,train_1.tif,0
2,train_2.tif,0
3,train_3.tif,0
4,train_4.tif,0
5,train_5.tif,0
6,train_6.tif,0
7,train_7.tif,0
8,train_8.tif,1
9,train_9.tif,0


In [25]:
df.flag.value_counts()

0    283740
1     12442
Name: flag, dtype: int64

In [31]:
df.set_index('file_name', inplace=True)

In [21]:
file_names = []
for file_name in tqdm(os.listdir(DATA_DIR.joinpath('train'))):
    if os.path.splitext(file_name)[1] != '.tif':
        continue
    file_names.append(file_name)

100%|██████████| 111836/111836 [00:00<00:00, 514377.46it/s]


In [24]:
file_names = np.array(file_names)

In [37]:
df = df.loc[file_names]

In [38]:
X = df.index.values
y = df.flag.values

In [39]:
X.shape

(111836,)

In [40]:
y.shape

(111836,)

In [41]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, random_state=216, stratify=y)

In [42]:
tar_dir = 'train'
X_ = X_train
y_ = y_train
for file_name, flag in tqdm(zip(X_, y_)):
    src = DATA_DIR.joinpath('train').joinpath(file_name)
    dst = DATA_DIR.joinpath('train2').joinpath(tar_dir).joinpath('0' if flag == 0 else '1').joinpath(file_name)
    shutil.copyfile(src, dst)

78285it [01:21, 957.18it/s]


In [43]:
tar_dir = 'valid'
X_ = X_valid
y_ = y_valid
for file_name, flag in tqdm(zip(X_, y_)):
    src = DATA_DIR.joinpath('train').joinpath(file_name)
    dst = DATA_DIR.joinpath('train2').joinpath(tar_dir).joinpath('0' if flag == 0 else '1').joinpath(file_name)
    shutil.copyfile(src, dst)

33551it [00:34, 963.12it/s]


In [19]:
batch_size = 200
nb_epoch = 9

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = DATA_DIR.joinpath('train2').joinpath('train')
validation_data_dir = DATA_DIR.joinpath('train2').joinpath('valid')
log_dir = LOG_DIR.joinpath('{:%Y-%m-%d%H%M%S}'.format(datetime.datetime.now()))
nb_train_samples = 78285
nb_validation_samples = 33551

init_shape = (None, 7, img_width, img_height) if K.image_dim_ordering() == 'th' else (None, 32, 32, 7)

def auc(y_true, y_pred) :
    score = tf.py_func( lambda y_true, y_pred : roc_auc_score(y_true, y_pred, average='macro', sample_weight=None).astype('float32'),
                        [y_true, y_pred],
                        'float32',
                        stateful=False,
                        name='sklearnAUC' )
    return score

# For WRN-16-8 put N = 2, k = 8
# For WRN-28-10 put N = 4, k = 10
# For WRN-40-4 put N = 6, k = 4
model = wrn.create_wide_residual_network(init_shape, nb_classes=2, N=2, k=8, dropout=0.30)

model.summary()
plot_model(model, "WRN-16-8.png", show_shapes=False)

adam = optimizers.Adam(lr=0.001)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc', auc])
print("Finished compiling")

# model.load_weights("weights/WRN-16-8 Weights.h5")
# print("Model loaded.")

cpcb = callbacks.ModelCheckpoint("weights/WRN-16-8 Weights.h5",
                                 monitor="val_acc",
                                 save_best_only=True,
                                 verbose=1)

tbcb = callbacks.TensorBoard(log_dir='./logs',
                             histogram_freq=0,
                             batch_size=batch_size,
                             write_graph=True,
                             write_grads=False,
                             write_images=True,
                             embeddings_freq=0,
                             embeddings_layer_names=None,
                             embeddings_metadata=None)

train_datagen = ImageDataGenerator(rotation_range=10,
                                   samplewise_center=True,
                                   samplewise_std_normalization=True,
                                   width_shift_range=5./32,
                                   height_shift_range=5./32,
                                   horizontal_flip=True,
                                   vertical_flip=True,)

test_datagen = ImageDataGenerator(samplewise_center=True,
                                  samplewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    color_mode='landsat',
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    color_mode='landsat',
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=nb_epoch,
    callbacks=[cpcb, tbcb],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    workers=4)

Wide Residual Network-21-8 created.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 7)    0                                            
__________________________________________________________________________________________________
conv2d_67 (Conv2D)              (None, 32, 32, 16)   1024        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_58 (BatchNo (None, 32, 32, 16)   64          conv2d_67[0][0]                  
__________________________________________________________________________________________________
activation_58 (Activation)      (None, 32, 32, 16)   0           batch_normalization_58[0][0]     
_________________________________________________________________________

Finished compiling
Found 78285 images belonging to 2 classes.
Found 33551 images belonging to 2 classes.
Epoch 1/9
391/391 [==============================] - 4816s 12s/step - loss: 0.6735 - acc: 0.9572 - auc: 0.5036 - val_loss: 0.6732 - val_acc: 0.9582 - val_auc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.95823, saving model to weights/WRN-16-8 Weights.h5
Epoch 2/9
 19/391 [>.............................] - ETA: 1:05:02 - loss: 0.6574 - acc: 0.9592 - auc: 0.5000

KeyboardInterrupt: 

In [9]:
testX = validation_generator.next()

In [10]:
testX

(array([[[[ 2.45895058e-01, -1.04769059e-01, -5.53790152e-01, ...,
            1.68917716e+00,  4.54369128e-01, -5.91208577e-01],
          [ 2.23978549e-01, -1.54482111e-01, -6.09383285e-01, ...,
            1.47535765e+00,  6.14756644e-02, -8.23737383e-01],
          [ 2.16494858e-01, -1.57154858e-01, -6.16866946e-01, ...,
            1.69505727e+00,  1.06912322e-01, -8.08235466e-01],
          ...,
          [ 2.19702154e-01, -1.61965802e-01, -6.02434158e-01, ...,
            1.57745647e+00,  1.91905603e-01, -7.51038730e-01],
          [ 1.96716547e-01, -1.69984028e-01, -5.94415903e-01, ...,
            2.68824911e+00,  2.94027273e-02, -8.86279643e-01],
          [ 1.81749180e-01, -2.07937002e-01, -6.55889034e-01, ...,
            2.44663310e+00, -3.27675968e-01, -1.10491014e+00]],
 
         [[ 2.56051481e-01, -9.03362334e-02, -5.39891899e-01, ...,
            1.40479720e+00,  6.88501596e-01, -4.55433160e-01],
          [ 2.34134972e-01, -1.35238349e-01, -6.02968693e-01, ...,
     

In [12]:
yPred = model.predict(testX[0])
yTrue = testX[1]

In [13]:
def IOU(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    p_true_index = np.where(y_true==1)[0]
    p_pred_index = np.where(y_pred==1)[0]
    union = set(p_true_index).union(set(p_pred_index))
    intersection = set(p_true_index).intersection(set(p_pred_index))
    if len(union)==0:
        return 0
    else:
        return len(intersection)/len(union)

In [14]:
roc_auc_score(yTrue, yPred)

0.5

In [15]:
IOU(yTrue, yPred)

0.065

In [16]:
yTrue

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [17]:
yPred

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [18]:
train_generator.next()

(array([[[[ 1.71127093e+00,  1.15129018e+00,  2.33081117e-01, ...,
           -6.26450419e-01, -9.93371844e-01, -1.10010290e+00],
          [ 1.71116734e+00,  1.15132594e+00,  2.33166903e-01, ...,
           -6.26314640e-01, -9.93289649e-01, -1.10006356e+00],
          [ 1.71116734e+00,  1.15132594e+00,  2.33166903e-01, ...,
           -6.26314640e-01, -9.93289649e-01, -1.10006356e+00],
          ...,
          [ 1.68003535e+00,  1.12148356e+00,  1.87691197e-01, ...,
           -6.65549159e-01, -1.00790989e+00, -1.10997069e+00],
          [ 1.69093239e+00,  1.11959255e+00,  2.05737814e-01, ...,
           -6.48453116e-01, -9.83779728e-01, -1.09385347e+00],
          [ 1.68281817e+00,  1.12197900e+00,  2.02396661e-01, ...,
           -6.51794255e-01, -1.00955439e+00, -1.10387707e+00]],
 
         [[ 1.72511292e+00,  1.14651716e+00,  2.21625715e-01, ...,
           -6.44588172e-01, -1.00434995e+00, -1.10535324e+00],
          [ 1.72339678e+00,  1.14710891e+00,  2.23045990e-01, ...,
     

In [20]:
model.optimizer.lr

<tf.Variable 'Adam/lr:0' shape=() dtype=float32_ref>

In [21]:
sess = K.get_session()

In [22]:
sess.run(model.optimizer.lr)

0.01

In [24]:
np.float32(0)

0.0